# KIPD: From Feasibility to Unconstrained Minimization

This notebook provides the code for the blog post *"KIPD: From Feasibility to Unconstrained Minimization"*. We use `cvxpy` to implement the four different semidefinite programming (SDP) formulations for finding the maximum lower bound $\gamma$ such that $p(x) - \gamma$ is a sum of squares (SOS).

We will focus on finding the global minimum of the univariate polynomial:
$$ p(x) = (x^2 - 2)^2 + 1 = x^4 - 4x^2 + 5 $$
The minimum of this polynomial is clearly 1, which occurs at $x = \pm\sqrt{2}$. Since all univariate non-negative polynomials are SOS, we expect our optimization to find $\gamma^* = 1$.

The monomial basis for a degree-4 polynomial is $\mathbf{v}(x) = [1, x, x^2]^\top$.

In [ ]:
import cvxpy as cp
import numpy as np
import time

# Set numpy print options for better readability
np.set_printoptions(precision=4, suppress=True)

## Setup: Defining the Problem Matrices

First, we define the matrices $\mathbf{A}_i$, $\mathbf{B}_j$, and the function for $\mathbf{Y}(\mathbf{p})$. These are the same building blocks used in the SOS feasibility problem.

In [ ]:
# The coefficient-matching matrices A_i for the constraint <X, A_i> = p_i
# A_0 corresponds to the constant term.
A = [
    np.array([[1, 0, 0], [0, 0, 0], [0, 0, 0]]), # A_0 for p_0
    np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]]), # A_1 for p_1
    np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]]), # A_2 for p_2
    np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]]), # A_3 for p_3
    np.array([[0, 0, 0], [0, 0, 0], [0, 0, 1]])  # A_4 for p_4
]

# The matrix B_1 for the null space of the coefficient-matching operator
B = [
    np.array([[0, 0, -0.5], [0, 1, 0], [-0.5, 0, 0]])
]

# A function to construct the matrix Y(p) from the polynomial coefficients p
# This Y(p) represents a particular solution to the system <X, A_i> = p_i
# This is not the only solution!
def get_Y(p_coeffs):
    p0, p1, p2, p3, p4 = p_coeffs
    return np.array([
        [p0,    p1/2, 0],
        [p1/2,  p2,   p3/2],
        [0,     p3/2, p4]
    ])

In [ ]:
from utils import svec, smat

# Automated way to get the list of B matrices by finding the null space of the coefficient-matching operator
n = A[0].shape[0]

data_matrix = np.empty((100, int(n * (n + 1) / 2)))
for i in range(100):
    x = np.random.rand()
    Xi = np.sum(np.stack([x**k * Ak[:, :, None] for k, Ak in enumerate(A)], axis=2), axis=2)[:, :, 0]
    data_matrix[i, :] = svec(Xi)

U, S, Vh = np.linalg.svd(data_matrix, full_matrices=False)
null_idx = S < 1e-10
basis = Vh[null_idx, :] 

B_list = []
for i in range(basis.shape[0]):
    np.testing.assert_almost_equal(data_matrix @ basis[i, :], 0.0)
    B_list.append(smat(basis[i, :]))
print(f"Found {len(B_list)} basis matrices for the null space:")
print(B_list)

## The Four Optimization Formulations

In [ ]:
def solve_primal_kernel_min(p_coeffs, A_matrices):
    """Solves the (P-K) Primal Kernel Optimization Problem."""
    X = cp.Variable((3, 3), symmetric=True)
    gamma = cp.Variable()
    p0, p1, p2, p3, p4 = p_coeffs
    
    constraints = [
        cp.trace(A_matrices[0] @ X) + gamma == p0,
        cp.trace(A_matrices[1] @ X) == p1,
        cp.trace(A_matrices[2] @ X) == p2,
        cp.trace(A_matrices[3] @ X) == p3,
        cp.trace(A_matrices[4] @ X) == p4,
        X >> 0
    ]
    
    problem = cp.Problem(cp.Maximize(gamma), constraints)
    start_time = time.time()
    problem.solve(solver=cp.SCS)
    end_time = time.time()
    
    return gamma.value, X.value, problem.status, end_time - start_time

def solve_primal_image_min(p_coeffs, A_matrices, B_matrices):
    """Solves the (P-I) Primal Image Optimization Problem."""

    assert len(B_matrices) == 1, "more than 1 B matrices currently not supported" 
    s = cp.Variable(len(B_matrices))
    gamma = cp.Variable()
    Y_p = get_Y(p_coeffs)
    
    constraint = (Y_p - gamma * A_matrices[0] + s[0] * B_matrices[0] >> 0)
    
    problem = cp.Problem(cp.Maximize(gamma), [constraint])
    start_time = time.time()
    problem.solve(solver=cp.SCS)
    end_time = time.time()
    
    X_sol = Y_p - gamma.value * A_matrices[0] + s.value[0] * B_matrices[0]
        
    return gamma.value, X_sol, problem.status, end_time - start_time

def solve_dual_kernel_min(p_coeffs, A_matrices):
    """Solves the (D-K) Dual Kernel Optimization Problem."""
    lambda_vars = cp.Variable(len(p_coeffs))
    
    constraints = [
        lambda_vars[0] == 1,
        cp.sum([lambda_vars[i] * A_matrices[i] for i in range(len(p_coeffs))]) >> 0
    ]
    
    objective = cp.Minimize(lambda_vars @ p_coeffs)
    problem = cp.Problem(objective, constraints)
    start_time = time.time()
    problem.solve(solver=cp.SCS)
    end_time = time.time()

    H_value = cp.sum([lambda_vars[i].value * A_matrices[i] for i in range(len(p_coeffs))])
    return problem.value, H_value, problem.status, end_time - start_time

def solve_dual_image_min(p_coeffs, A_matrices, B_matrices):
    """Solves the (D-I) Dual Image Optimization Problem."""
    X = cp.Variable((3, 3), symmetric=True)
    Y_p = get_Y(p_coeffs)
    
    constraints = [
        cp.trace(A_matrices[0].T @ X) == 1,
        cp.trace(B_matrices[0].T @ X) == 0,
        X >> 0
    ]
    
    objective = cp.Minimize(cp.trace(Y_p.T @ X))
    problem = cp.Problem(objective, constraints)
    start_time = time.time()
    problem.solve(solver=cp.SCS)
    end_time = time.time()

    return problem.value, X.value, problem.status, end_time - start_time

## Example: Finding the Minimum of $p(x) = x^4 - 4x^2 + 5$

The coefficient vector for this polynomial is $\mathbf{p} = [5, 0, -4, 0, 1]$. We expect to find that the optimal `γ` is 1.

In [ ]:
p_coeffs = np.array([5., 0., -4., 0., 1.])

print("--- Solving SOS Optimization Problem ---")
print(f"p(x) = {p_coeffs[0]} + {p_coeffs[2]}x^2 + {p_coeffs[4]}x^4\n")

# (P-K)
gamma_pk, X_pk, status, t = solve_primal_kernel_min(p_coeffs, A)
eigs = np.linalg.eigvalsh(X_pk)
print(f"(P-K) Primal Kernel: Status='{status}', Time={t:.4f}s")
print(f"Optimal gamma = {gamma_pk:.4f}")
print(f"Gram matrix X:\n{X_pk}, eigenvalues: {eigs}\n,")

# (P-I)
gamma_pi, X_pi, status, t = solve_primal_image_min(p_coeffs, A, B)
eigs = np.linalg.eigvalsh(X_pi)
print(f"(P-I) Primal Image:  Status='{status}', Time={t:.4f}s")
print(f"Optimal gamma = {gamma_pi:.4f}")
print(f"Gram matrix X:\n{X_pi}, eigenvalues: {eigs}\n")

# (D-K)
gamma_dk, L_dk, status, t = solve_dual_kernel_min(p_coeffs, A)
eigs = np.linalg.eigvalsh(L_dk)
print(f"(D-K) Dual Kernel:   Status='{status}', Time={t:.4f}s")
print(f"Optimal objective (gamma) = {gamma_dk:.4f}")
print(f"Dual matrix H:\n{L_dk}, eigenvalues: {eigs}\n")

# (D-I)
gamma_di, L_di, status, t = solve_dual_image_min(p_coeffs, A, B)
eigs = np.linalg.eigvalsh(L_di)
print(f"(D-I) Dual Image:    Status='{status}', Time={t:.4f}s")
print(f"Optimal objective (gamma) = {gamma_di:.4f}")
print(f"Dual matrix H:\n{L_di}, eigenvalues: {eigs}\n")

## Analysis of the Results

As expected, all four formulations correctly find the optimal value $\gamma = 1.0$.

We also notice that the solution the the primal formulations are of rank one, and that we have the factorizations

$$
\begin{bmatrix}
4 & 0 & -2 \\
0 & 0 & 0 \\
-2 & 0 & 1
\end{bmatrix}
= \begin{bmatrix}
-2 \\ 
0 \\
1
\end{bmatrix}\begin{bmatrix} -2 & 0 & 1\end{bmatrix}
$$

The solution of the dual formulations are of rank two, and it is therefore not straight forward to extract a rank-one solution. This is to be expected because our original problem has two optimal solutions. Let's perturb the problem so that only one of the solutions is optimal.

This is easily achieved by adding a (very small) term to the problem.

In [ ]:
p_coeffs = np.array([5., 1e-3, -4., 0., 1.])

print("--- Solving SOS Optimization Problem ---")
print(f"p(x) = {p_coeffs[0]} + {p_coeffs[2]}x^2 + {p_coeffs[4]}x^4\n")

# (P-K)
gamma_pk, X_pk, status, t = solve_primal_kernel_min(p_coeffs, A)
eigs = np.linalg.eigvalsh(X_pk)
print(f"(P-K) Primal Kernel: Status='{status}', Time={t:.4f}s")
print(f"Optimal gamma = {gamma_pk:.4f}")
print(f"Gram matrix X:\n{X_pk}, eigenvalues: {eigs}\n,")

# (D-K)
gamma_dk, L_dk, status, t = solve_dual_kernel_min(p_coeffs, A)
eigs = np.linalg.eigvalsh(L_dk)
print(f"(D-K) Dual Kernel:   Status='{status}', Time={t:.4f}s")
print(f"Optimal objective (gamma) = {gamma_dk:.4f}")
print(f"Dual matrix H:\n{L_dk}, eigenvalues: {eigs}\n")

## Analysis of the Results

Now, the dual solution has rank one! And it factorizes (approximately, because of the 1e-3 term that we added), as follows:

$$
\begin{bmatrix}
1 & -\sqrt(2) & 2 \\
-\sqrt{2} & 2 & -2\sqrt(2) \\
2 & -2\sqrt(2) & 4
\end{bmatrix}
= \begin{bmatrix}
1 \\ 
-\sqrt{2} \\
2 
\end{bmatrix}\begin{bmatrix} 1 & \sqrt{2} & 2 \end{bmatrix}
$$

As expected, now we have correctly identified the global solution $x=-\sqrt{2}$.